In [1]:
%pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import catboost as cb

In [4]:
test = pd.read_csv('/content/drive/MyDrive/SAMOLET_CASE/data/X_test.csv').set_index('dt')
model = cb.CatBoostRegressor()
model.load_model('/content/drive/MyDrive/SAMOLET_CASE/model/model.cbm')

In [5]:
preds = model.predict(test)

In [6]:
preds_df = pd.DataFrame([test.index,preds]).T
preds_df.columns = ['dt','Цена на арматуру']

In [8]:
# "коэффициент доверия"
WEEK_OF_CONFIDENCE = 5
# максимально допустимый период для предсказаний, задается как множитель к коэффициенту доверия
PERIOD = 2

# значение основано на метрике модели ML, диапазон данных
INTERVAL = 600
COEF_ = 1

MAX_PUSHASE = 10

In [10]:
i = 0
while i < preds_df.shape[0]:
  # тут должно быть переобучение модели на текущую дату
  # ___________________
  price_on_date = preds_df[i:i+WEEK_OF_CONFIDENCE].reset_index(drop=True)
  # ___________________
  price_on_date = price_on_date['Цена на арматуру']
  try:
    # если в заданный интервал есть снижение, то покупаем на период до снижения
    res = list(price_on_date[0] > price_on_date[1:]).index(1) + 1
    preds_df.loc[i, 'res'] = res
    i += res
  except:
    # если в заданный интервал снижения нет - берем больший период времени.
    # т.к. при предсказании сильно вперед может накапливаться ошибка - будем рассматривать данные с учетом ошибки

    # тут переобучение модели на бОльший период
    # ______________________
    price_on_date = preds_df[i:(i+WEEK_OF_CONFIDENCE)*PERIOD].reset_index(drop=True)
    # ______________________

    ## пока что задано хардкодингом.
    k = WEEK_OF_CONFIDENCE

    # чем дальше предсказание, тем точность ниже. искуственно "понижаем"
    # (берем нижнюю границу) для оценки закупки на большой период
    # т.е. пытаемся прикинуть закупку при "лучшем варианте"
    for j in range(k, k*PERIOD + 1, 2):
       price_on_date.loc[(k+j):(k+j+2), 'Цена на арматуру'] -= (j+1) * COEF_ * INTERVAL

    price_on_date = price_on_date['Цена на арматуру']
    try:
      res = list(price_on_date[0] > price_on_date[1:]).index(1) + 1
      preds_df.loc[i, 'res'] = res
      i += res
    except:
      preds_df.loc[i, 'res'] = MAX_PUSHASE
      i += MAX_PUSHASE

In [25]:
preds_df = preds_df.fillna(0)
preds_df.columns = ['dt','Цена на арматуру', 'Объем']
preds_df['dt'] = pd.to_datetime(preds_df['dt'])


In [26]:
def decision_prices(test):

    test = test.set_index('dt')
    tender_price = test['Цена на арматуру']
    decision = test['Объем']
    start_date = test.index.min()
    end_date = test.index.max()

    _results = []
    _active_weeks = 0
    for report_date in pd.date_range(start_date, end_date, freq='W-MON'):
        if _active_weeks == 0:  # Пришла пора нового тендера
            _fixed_price = tender_price.loc[report_date]
            _active_weeks = int(decision.loc[report_date])
        _results.append(_fixed_price)
        _active_weeks += -1
    cost = sum(_results)
    return cost # Возвращаем затраты на периоде

In [27]:
decision_prices(preds_df)

453258.97102115117